In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
import os

In [23]:
train = pd.read_csv('./tensorflow-great-barrier-reef/train.csv')
train['pos'] = train.annotations != '[]'
fold = 1

In [24]:
HOME_DIR = "./"
DATASET_PATH = os.path.join('yolo_data', f'fold{str(fold)}', 'images')
LABELS_PATH = os.path.join('yolo_data', f'fold{str(fold)}', 'labels')

os.makedirs(os.path.join(HOME_DIR, 'yolo_data', f'fold{str(fold)}'))

for path in [DATASET_PATH, LABELS_PATH]:
    os.makedirs(os.path.join(HOME_DIR, path))
    os.makedirs(os.path.join(HOME_DIR, path, 'train'))
    os.makedirs(os.path.join(HOME_DIR, path, 'val'))

In [25]:
# sharp_filter = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
# def gamma_enhance(img, gamma=1.0):
#     lookUpTable = np.empty((1,256), np.uint8)
#     for i in range(256):
#         lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
#     return cv2.LUT(img, lookUpTable)

In [26]:
# # Preprocessed Images

# for i in tqdm(range(len(train))):
#     row = train.loc[i]
#     img = cv2.imread(row.image_path)
#     # img = gamma_enhance(cv2.filter2D(img, -1, sharp_filter), gamma=2)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     if row.video_id != fold:
#         cv2.imwrite(f'{HOME_DIR}{DATASET_PATH}/train/{row.image_id}.jpg', img)
#         # copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/train/{row.image_id}.jpg')
#     else:
#         cv2.imwrite(f'{HOME_DIR}{DATASET_PATH}/val/{row.image_id}.jpg', img)
#         # copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/val/{row.image_id}.jpg') 

In [28]:
annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'./tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)